In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/newninaprodb4/ninaprodb44test.pkl
/kaggle/input/newninaprodb4/ninaprodb44train.pkl
/kaggle/input/datasetfornina/ninaprodb1train.pkl
/kaggle/input/datasetfornina/ninaprodb1test.pkl
/kaggle/input/emg-data-for-gestures-data-set/EMG/Edited_label/1.csv
/kaggle/input/emg-data-for-gestures-data-set/EMG/Edited_label/3.csv
/kaggle/input/emg-data-for-gestures-data-set/EMG/Edited_label/5.csv
/kaggle/input/emg-data-for-gestures-data-set/EMG/Edited_label/2.csv
/kaggle/input/emg-data-for-gestures-data-set/EMG/Edited_label/4.csv
/kaggle/input/emg-data-for-gestures-data-set/EMG/Edited_label/7.csv
/kaggle/input/emg-data-for-gestures-data-set/EMG/Edited_label/0.csv
/kaggle/input/emg-data-for-gestures-data-set/EMG/Edited_label/6.csv
/kaggle/input/emg-data-for-gestures-data-set/EMG/Raw_label/1.csv
/kaggle/input/emg-data-for-gestures-data-set/EMG/Raw_label/3.csv
/kaggle/input/emg-data-for-gestures-data-set/EMG/Raw_label/5.csv
/kaggle/input/emg-data-for-gestures-data-set/EMG/Raw_label/2.csv
/k

In [2]:
import numpy as np
import os
from tensorflow.keras.layers import TimeDistributed, Conv1D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Activation

# ... (other code) ...


import random
#from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import math

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras import layers, Sequential, optimizers, Input, Model

**WORKED CODE HERE**

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import gc
import random
import os

import matplotlib.pyplot as plt
import json

In [6]:
epochs = 200
learning_rate = 1e-3
batch_size = 16
method = "default"
dataset_type = 1

In [7]:
import tensorflow as tf

from tensorflow.keras.activations import tanh
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Reshape

from tensorflow.keras.layers import LSTM, Bidirectional, Dropout
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM, Bidirectional, Dropout, Input, Conv1D, MaxPooling1D,
    Flatten, Dense, ReLU, TimeDistributed, Reshape, BatchNormalization
)
from tensorflow.keras.optimizers.schedules import CosineDecayRestarts
from tensorflow.keras import regularizers, initializers

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import pywt
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical


class Nina1Dataset(tf.keras.utils.Sequence):
    def __init__(self, dataframe, batch_size):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.scaler = StandardScaler()
        self.scaler.fit(np.concatenate(self.dataframe['emg'].tolist()))
        
    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))

    def __getitem__(self, idx):
        batch_data = self.dataframe[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_input_data = []
        batch_labels = []

        for i, target_row in batch_data.iterrows():
            data = target_row['emg'][:500]

            # Zero-Padding
            if len(data) < 500:
                data = np.concatenate((data, np.zeros((500 - len(data), 10))), axis=0)
            coeffs = pywt.wavedec(data, 'sym8')  # Using 3 decomposition levels as an example
            coeffs[1:] = (pywt.threshold(c, value=0.5, mode='soft') for c in coeffs[1:])
            data = pywt.waverec(coeffs, 'sym8')
            data = self.scaler.transform(data)
            
            # Division data by time-segment
            #input_data = np.transpose(data.reshape((25, 20, 10)), (0, 2, 1))
            input_data = data.reshape((25, 20, 10))
            label = target_row['stimulus']
            #label = to_categorical(label)
            batch_input_data.append(input_data)
            batch_labels.append(label)

        # Check if the batch size is smaller than the desired batch_size
        if len(batch_data) < self.batch_size:
            # Create a dummy batch with all elements set to zero
            dummy_input_data = np.zeros((self.batch_size,) + input_data.shape, dtype=np.float32)
            dummy_labels = np.zeros((self.batch_size,), dtype=np.int32)
            dummy_input_data[:len(batch_input_data)] = np.array(batch_input_data)
            dummy_labels[:len(batch_labels)] = np.array(batch_labels)
            dummy_labels = to_categorical(dummy_labels,num_classes=52)  # Convert labels to one-hot encoding
            
            return dummy_input_data, dummy_labels
        
        batch_labels = to_categorical(batch_labels,num_classes=52)  # Convert labels to one-hot encoding
        
        return np.array(batch_input_data), np.array(batch_labels)





# Parameters
batch_size =16
train_dir = '/kaggle/input/datasetfornina/ninaprodb1train.pkl'
test_dir = '/kaggle/input/datasetfornina/ninaprodb1test.pkl'

# Set up dataset

train = pd.read_pickle(train_dir)
eval_data = pd.read_pickle(test_dir)

# Load train data
train_data = pd.read_pickle(train_dir)

# Split data into train and validation sets
#train_data, val_data = train_test_split(train_data, test_size=0.3, random_state=21)

train_data = train_data.sample(frac=1).reset_index(drop=True)
#val_data = val_data.sample(frac=1).reset_index(drop=True)
eval_data = eval_data.sample(frac=1).reset_index(drop=True)

# Create train, test and validation datasets
train_dataset = Nina1Dataset(train_data, batch_size=batch_size)
#val_dataset= Nina1Dataset(val_data, batch_size=batch_size)
test_dataset= Nina1Dataset(eval_data, batch_size=batch_size)






In [9]:
# Create x_train, y_train from train_dataset
x_train = []
y_train = []

for i in range(len(train_dataset)):
    batch_x, batch_y = train_dataset[i]
    x_train.extend(batch_x)
    y_train.extend(batch_y)

x_train = np.array(x_train)
y_train = np.array(y_train)

# Create x_test, y_test from test_dataset
x_test = []
y_test = []

for i in range(len(test_dataset)):
    batch_x, batch_y = test_dataset[i]
    x_test.extend(batch_x)
    y_test.extend(batch_y)

x_test = np.array(x_test)
y_test = np.array(y_test)

# Print the shapes of the datasets
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)




x_train shape: (9840, 25, 20, 10)
y_train shape: (9840, 52)
x_test shape: (4224, 25, 20, 10)
y_test shape: (4224, 52)


In [10]:
import tensorflow as tf
from tensorflow.keras.layers import (
    LSTM, Bidirectional, Dropout, Input, Conv1D, MaxPooling1D,
    Flatten, Dense, Activation, TimeDistributed, BatchNormalization
)
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model
import numpy as np
def cnn(x):
    #print(x.shape)
    #x = Reshape((25, 20,10))(x)  
    x = TimeDistributed(Conv1D(filters=64, kernel_size=9, padding='same', kernel_initializer="he_normal", strides=2, kernel_regularizer=l2(1e-04)))(x)
    x = TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None))(x)
    x = TimeDistributed(MaxPooling1D(pool_size=8, strides=2))(x)
    x = TimeDistributed(Activation('tanh'))(x)
    x = TimeDistributed(Conv1D(filters=64, kernel_size=5, padding="same", kernel_initializer="he_normal", strides=2, kernel_regularizer=l2(1e-04)))(x)
    x = TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None))(x)
    x = TimeDistributed(Activation('tanh'))(x)
    x = TimeDistributed(Dropout(0.2093))(x)
    x = TimeDistributed(Conv1D(filters=64, kernel_size=5, padding="same", kernel_initializer="he_normal", strides=2, kernel_regularizer=l2(1e-04)))(x)
    x = TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None))(x)
    x = TimeDistributed(Activation('tanh'))(x)
    x = TimeDistributed(Dropout(0.2093))(x)
    x = TimeDistributed(Conv1D(filters=64, kernel_size=3, padding="same", kernel_initializer="he_normal", strides=2, kernel_regularizer=l2(1e-04)))(x)
    x = TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None))(x)
    x = TimeDistributed(Activation('tanh'))(x)
    x = TimeDistributed(Dropout(0.2093))(x)
    x = TimeDistributed(Dropout(0.5))(x)
    x = TimeDistributed(Activation('relu'))(x)
    x = TimeDistributed(Flatten())(x)
    return x
def Bi_LSTMModel(input_shape,x):
    #model = Sequential()
    # Hidden dimensions
    hidden_dim = 200
    print(x.shape)
    print(11)
    x = Bidirectional(LSTM(hidden_dim, return_sequences=True), input_shape=input_shape)(x)
    x = Dropout(0.3)(x)
    print(x.shape)
    print(11)
    x = Bidirectional(LSTM(hidden_dim, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    print(x.shape)
    print(11)
    #x = Dropout(0.2093)(x)
    x = Flatten()(x)
    # ( ,10000)
    return x
import tensorflow as tf
from tensorflow.keras.layers import (
    LSTM, Bidirectional, Dropout, Input, Conv1D, MaxPooling1D,
    Flatten, Dense, Activation, TimeDistributed, BatchNormalization
)
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model
import numpy as np

# Define the teacher model
def create_teacher_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = inputs
    x = cnn(x)  # Use the cnn function from your code
    x = Bi_LSTMModel(x.shape[1:], x)  # Use the Bi_LSTMModel function from your code
    x = Dense(512, activation='tanh')(x)
    x = BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    teacher_model = Model(inputs=inputs, outputs=output_layer)
    return teacher_model

# Define the student model (smaller model)
def create_student_model(input_shape, num_classes):
    # Define an additional input for teacher model's predictions
    teacher_predictions = Input(shape=(num_classes,))

    inputs = Input(shape=input_shape)
    x = inputs
    x = cnn(x)  # Use the cnn function from your code
    x = Bi_LSTMModel(x.shape[1:], x)  # Use the Bi_LSTMModel function from your code
    x = Dense(256, activation='tanh')(x)  # Smaller hidden layer
    x = BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)(x)
    
    output_layer = Dense(num_classes, activation='softmax')(x)

    student_model = Model(inputs=[inputs, teacher_predictions], outputs=output_layer)
    return student_model

# Create teacher and student models
num_classes = 52
teacher_model = create_teacher_model((25, 20, 10), num_classes)
student_model = create_student_model((25, 20, 10), num_classes)



# Define the learning rate schedule
initial_learning_rate = 1e-3
decay_steps = 1000
decay_rate = 0.9
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps, decay_rate)

# Compile the teacher model
teacher_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule,
                                                         beta_1=0.9,
                                                         beta_2=0.999,
                                                         amsgrad=False),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

# Train the teacher model
teacher_model.fit(x_train, y_train, epochs=200, batch_size=16, validation_data=(x_test, y_test))

# Compile the student model with the learning rate schedule and knowledge distillation loss
student_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule,
                                                         beta_1=0.9,
                                                         beta_2=0.999,
                                                         amsgrad=False),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

# Train the student model using knowledge distillation
history = student_model.fit([x_train, teacher_model.predict(x_train)],
                            y_train,
                            epochs=200,
                            batch_size=16,
                            validation_data=([x_test, teacher_model.predict(x_test)], y_test),
                            verbose=1)


(None, 25, 64)
11
(None, 25, 400)
11
(None, 25, 400)
11
(None, 25, 64)
11
(None, 25, 400)
11
(None, 25, 400)
11
Epoch 1/200
615/615 [==============================] - 40s 25ms/step - loss: 3.6520 - accuracy: 0.1188 - val_loss: 2.9467 - val_accuracy: 0.2206
Epoch 2/200
615/615 [==============================] - 14s 23ms/step - loss: 2.7775 - accuracy: 0.2575 - val_loss: 2.3582 - val_accuracy: 0.3501
Epoch 3/200
615/615 [==============================] - 15s 24ms/step - loss: 2.4405 - accuracy: 0.3285 - val_loss: 2.1898 - val_accuracy: 0.4264
Epoch 4/200
615/615 [==============================] - 13s 22ms/step - loss: 2.1864 - accuracy: 0.3832 - val_loss: 1.8749 - val_accuracy: 0.4986
Epoch 5/200
615/615 [==============================] - 15s 24ms/step - loss: 2.0186 - accuracy: 0.4293 - val_loss: 1.5717 - val_accuracy: 0.5405
Epoch 6/200
615/615 [==============================] - 13s 22ms/step - loss: 1.8252 - accuracy: 0.4730 - val_loss: 1.4816 - val_accuracy: 0.5821
Epoch 7/200
615/61